In [1]:
!nvidia-smi

Sun May  5 03:40:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_ckpt = 'minhtoan/t5-small-wikilingua-vietnamese'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [9]:
import pandas as pd

# Đường dẫn tới file CSV
url = "https://huggingface.co/datasets/OpenHust/vietnamese-summarization/resolve/main/Kmeans_1024_new.csv?download=true"

# Đọc file CSV từ URL
df = pd.read_csv(url)

# Hiển thị 5 dòng đầu tiên để kiểm tra
print(df.head())


   Unnamed: 0                                           Document  \
0           0  Đây là một trong những nội dung tại văn bản vừ...   
1           1  Chiều 12/3 , ông Vũ Hùng Triều , Trưởng phòng ...   
2           2  Thoạt đầu nhìn vào bức ảnh , nếu không có dấu ...   
3           3  Khoảng 9h30 sáng 5/6 , anh Đào Nhật Tuấn ( 30 ...   
4           4  Liên minh Tự do Dân sự Mỹ ( ACLU ) đã nhận đượ...   

                                             Summary   Dataset  
0  Các quận , huyện , thị xã tuyên truyền bằng nh...  vietnews  
1  Cháu bé được phát hiện trong tình trạng chưa c...  vietnews  
2  Báo tuyết ( Panthera uncia ) được mệnh danh là...  vietnews  
3  Thấy con chim bị mắc bẫy rơi xuống hồ nước , a...  vietnews  
4  Loại " huyết thanh nói thật " mà CIA sử dụng đ...  vietnews  


In [ ]:
df['Document'][0]

'Đây là một trong những nội dung tại văn bản vừa được UBND TP Hà Nội ban hành về việc tăng cường công tác quản lý nuôi , giết mổ , kinh doanh và sử dụng thịt chó , mèo trên địa bàn .Theo đó , các sở , ngành trên địa bàn phải vào cuộc ngay để hướng tới thay đổi thói quen của người dân khi dùng chó , mèo làm thực phẩm .Gây phản cảm với du khách , người nước ngoàiCũng trong văn bản này , UBND TP Hà Nội thừa nhận rằng việc kinh doanh , giết mổ và sử dụng thịt chó , mèo tại Hà Nội thời gian qua đã tạo ra những hình ảnh phản cảm đối với khách tham quan du lịch , người nước ngoài đang sinh sống và làm việc tại Hà Nội , gây ảnh hưởng tới hình ảnh của một thủ đô " văn minh , hiện đại " .Trong thực tế , theo ghi nhận của Tuổi Trẻ tại phố Tam Trinh , ngay đoạn đầu cầu Mai Động ( quận Hoàng Mai ) , một đoạn phố dài với gần chục cửa hàng buôn bán thịt chó sống , nhà hàng phục vụ các món chế biến từ thịt chó vẫn hoạt động tấp nập nhiều năm nay .Chị Th . , một chủ sạp bán thịt chó sống ở đây , cho ha

In [10]:
from datasets import DatasetDict, Dataset

# Chuyển đổi dữ liệu thành định dạng Dataset
dataset_dict = DatasetDict()

# Tách dữ liệu thành tập huấn luyện, tập kiểm tra và tập validation
train_df = df.sample(frac=0.8, random_state=42)
val_test_df = df.drop(train_df.index)
val_df = val_test_df.sample(frac=0.5, random_state=42)
test_df = val_test_df.drop(val_df.index)

# Tạo Dataset cho mỗi tập
train_dataset = Dataset.from_pandas(train_df.rename(columns={"Unnamed: 0": "id"}))
val_dataset = Dataset.from_pandas(val_df.rename(columns={"Unnamed: 0": "id"}))
test_dataset = Dataset.from_pandas(test_df.rename(columns={"Unnamed: 0": "id"}))

# Gán features
train_dataset = train_dataset.map(lambda example: {'id': example['id'], 'dialogue': example['Document'], 'summary': example['Summary']})
val_dataset = val_dataset.map(lambda example: {'id': example['id'], 'dialogue': example['Document'], 'summary': example['Summary']})
test_dataset = test_dataset.map(lambda example: {'id': example['id'], 'dialogue': example['Document'], 'summary': example['Summary']})

# Đưa các dataset vào DatasetDict
dataset_dict["train"] = train_dataset
dataset_dict["validation"] = val_dataset
dataset_dict["test"] = test_dataset

print(dataset_dict)

Map:   0%|          | 0/8522 [00:00<?, ? examples/s]

Map:   0%|          | 0/1065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1065 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'Document', 'Summary', 'Dataset', '__index_level_0__', 'dialogue', 'summary'],
        num_rows: 8522
    })
    validation: Dataset({
        features: ['id', 'Document', 'Summary', 'Dataset', '__index_level_0__', 'dialogue', 'summary'],
        num_rows: 1065
    })
    test: Dataset({
        features: ['id', 'Document', 'Summary', 'Dataset', '__index_level_0__', 'dialogue', 'summary'],
        num_rows: 1065
    })
})


In [ ]:
# import string
# import re
# from pyvi import ViTokenizer # Để phân đoạn tiếng Việt
# from pyvi import ViSpacyTokenizer # Để phân đoạn tiếng Việt với spaCy
# from underthesea import word_tokenize # Để phân đoạn tiếng Việt với Underthesea

# # Hàm loại bỏ dấu câu
# def remove_punctuation(text):
#     return re.sub(r'[{}]'.format(string.punctuation), '', text)

# # Hàm chuyển đổi thành chữ thường
# def lowercase_conversion(text):
#     return text.lower()

# # Hàm loại bỏ stopword
# def remove_stopwords(text, stopwords):
#     words = text.split()
#     return ' '.join([word for word in words if word not in stopwords])

# # Hàm sửa lỗi chính tả
# def spelling_correction(text):
#     # Implement your spelling correction logic here
#     # You can use libraries like pyvi, underthesea, or other Vietnamese NLP libraries
#     return text

# # Ví dụ cách sử dụng các hàm trên
# text = "Đây là một ví dụ về việc loại bỏ dấu câu, chuyển đổi thành chữ thường, và loại bỏ stopword."
# # Loại bỏ dấu câu
# text_without_punctuation = remove_punctuation(text)
# # Chuyển đổi thành chữ thường
# text_lowercase = lowercase_conversion(text_without_punctuation)
# # Tách từ
# # Có thể sử dụng một trong các phương pháp phân đoạn sau:
# # Tách từ với ViTokenizer từ thư viện pyvi
# # text_tokenized = ViTokenizer.tokenize(text_lowercase)
# # Tách từ với ViSpacyTokenizer từ thư viện pyvi
# # text_tokenized = ' '.join(ViSpacyTokenizer.tokenize(text_lowercase))
# # Tách từ với Underthesea
# text_tokenized = ' '.join(word_tokenize(text_lowercase))

# print("Text after preprocessing:", text_tokenized)


In [11]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

  with tokenizer.as_target_tokenizer():
      target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True)

  return {
      'input_ids' : input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

In [12]:
dataset_samsum_pt = dataset_dict.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/8522 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1065 [00:00<?, ? examples/s]

In [ ]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'Document', 'Summary', 'Dataset', '__index_level_0__', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8522
})

In [16]:
#Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [18]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='small-wikilingua-vietnamese', num_train_epochs=3, warmup_steps=500,
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [19]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['train'],
                  eval_dataset=dataset_samsum_pt['validation'])

In [22]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=399, training_loss=1.9675285176824509, metrics={'train_runtime': 3272.8645, 'train_samples_per_second': 7.812, 'train_steps_per_second': 0.122, 'total_flos': 2.661252960768e+16, 'train_loss': 1.9675285176824509, 'epoch': 2.995776630689817})

In [23]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [24]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
score = calculate_metric_on_test_ds(
    dataset_samsum_pt['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'model'] )


100%|██████████| 5/5 [00:13<00:00,  2.69s/it]


,rouge1,rouge2,rougeL,rougeLsum
model,0.367287,0.083138,0.251823,0.250732


In [26]:
#Save model
model_pegasus.save_pretrained('/content/drive/MyDrive/pegasus model/pegasus-samsum-model')

In [27]:
## Save tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/pegasus model/tokenizer")

('/content/drive/MyDrive/pegasus model/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/pegasus model/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/pegasus model/tokenizer/spiece.model',
 '/content/drive/MyDrive/pegasus model/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/pegasus model/tokenizer/tokenizer.json')

In [28]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/pegasus model/tokenizer")

In [29]:
def truncate_sample(text):
  input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=False)
  if len(input_ids) > 1024:
      input_ids = input_ids[:1024]
      # Đảo ngược lại từ token thành văn bản
      return tokenizer.decode(input_ids, skip_special_tokens=True)
  else:
    return text

In [30]:
#Prediction

sample = dataset_samsum_pt["test"][1]

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}


sample_text = truncate_sample(sample["dialogue"])

reference = sample["summary"]


In [36]:
model_path = '/content/drive/MyDrive/pegasus model/pegasus-samsum-model'

In [37]:
pipe = pipeline("summarization", model=model_path,tokenizer=tokenizer)

In [38]:
##
print("Dialogue:")
print(sample["dialogue"])


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:
Theo WebMD dẫn nguồn Viện hàn lâm Hoa Kỳ , ngoài bài làm ở nhà , trẻ ở tuổi đến trường chỉ nên dành không quá một hay hai giờ với màn hình mỗi ngày .Giáo sư Craig Anderson - giáo sư về sinh lý học Đại học bang Iowa - nói : " Có rất nhiều tác động tiềm ẩn từ màn hình có hại đến trẻ sơ sinh tới thanh thiếu niên và cả người lớn còn trẻ " .Giáo sư Craig nói khi trẻ xem khá nhiều chương trình , chuyển nhanh chóng từ cảnh này qua cảnh khác , sau này chúng có thể thấy bối rối khi cần tập trung trong lớp .Còn theo bác sĩ David Hill - chủ tịch Hội đồng về truyền thông và thông tin Viện hàn lâm Nhi khoa Hoa Kỳ , trẻ dành quá nhiều thời gian trước màn hình có thể mắc những bệnh chứng khác nữa , như kém ngủ hay quá tăng cân .10 mẹo giúp trẻ em rời mắt khỏi các thiết bị , màn hình có ánh sáng xanh .1 Đừng cho các con bạn sở hữu máy tính bảng hay điện thoại thông minh .2 Hạn chế để máy vi tính và tivi ở một số chỗ thuận mắt trẻ , trẻ dễ tiện tay dùng .Ví dụ khi bạn để màn hình ở bếp , các 